#Outline of Project

##Dashboard as a Communication Tool

Here, we write some hypotheses and predictions based on work we've read in class about the efficacy of communication and rapid/accurate data transfer between agents to accomplish a task effexctively
 

##Dashboard as a Fire Modelling and Recommendation Tool

Here we outline the way our dashboard works to model a fire based on some data and inputs and the ways in which we can model extinguishing this fire and which team methods work better for this purpose

##Putting it all together

Combining these two components in the paper, we can essentialyl draw the conclusion that by improving communication and record-keeping between jumpers and dispatch, and by also being able to provide solid real-time recommendations, we can utilize multi-agent interactions coumpled with a powerful fire model to improve the effectiveness fo wildfire fighting operations by smokejumpers.

##Sequence of iterations to final product
Mine any necessary or valuable data
Inputs to dispatch:
   - Location of fire
   - Vegetation
       - Type
       - Flammability
       - Weighted avg of all types of fuel
       - Amount (volume) of fuel
   - Size/shape of fire
       - Mark more valuable/useful coordinates through which the fire passes
       - Main points of the fire
       - Where is the anchor point?
       - 2D/3D grid of where the fire is
   - Wind estimate
   - Conditions estimate (dry, wet, etc)
   - Rate of speed?
   - Sensitive burning areas
       - Cost estimates (if we're really fancy)

Find a model that we actually want to use so that we know which of these we need and **in what format** (ie how to we store shape and size of the fire?)

Outputs from dispatch:
   - Temporal estimate of what the fire will look like look like at given time intervals (30m/1hr/2hr)
   - Will the anchor move? Will it spread in a shape that requires more detailed information
   - Update the grid of the fire if it exists
   - Where to target the fire and how many people per target
   - Stupid extension stuff:
       - Calculate predicted cost? (if we're fancy)
       - Compute a danger evaluation (Scale 1-100 of urgency or something)
       - Calculate distance to base and approx flight time
       - Provide a resources recommendation (# of fighters, # of planes, amt of fuel, level of IC)
    
Create a class that collects inputs from dispatch

Create some function/class that can take 1-many variables and spt out some information about a fire

Dashboard that takes in user input and returns a fire/fire spread model

Graphical Dashboard

###Evaluation and means of testing

Given some model of the fire, most likely derived from the temperature paper, we want to make some prediction as to the best arrangement of smokejumpers to most efficiently handle the fire.

This can be done by making each fighter a hypothetical "cooling point" and modeling what happens to the fire, essentially determining how quickly it will be put out.

We can determine these cooling pts automatically by trying a bunch of combinations and finding the best one.

This doesn't take into account safety of firefighters, so we should implement a safety rating that is represented by a negative reward (anything to inform realistic landing areas).

On top of this, we can create 2-3 heuristic approaches to fire tackling (attack the anchor, attack the flanks, cluster together in one location, or spread out in and/or around the fire [email someone to find out possible approaches of attack])

Using these heuristic approaches, we can take a couple of verieties of fire (big, small, long, round, multi-point, etc [look into finding realistic approximations]), and perhaps even getting actual fires that were recorded in alaska and using their dimensions or any info we can get on them.

Then we can run the heuristic approaches on the different types of fires (based on our spread model) and evaluate how quickly the fire gets put out in each scenario and how far it spreads in the time it takes to put it out

###Timeline

####Coding component

[by Apr 23] Using this paper: http://arxiv.org/pdf/0709.0086.pdf, create a Python class that is able to take fire inputs (assuming we have them) and spit out some desired output. 

[by Apr 25] Using not real, toy model data, make sure this model works and simulate a cooling point to see if we can actually extinguish a fire

[by Apr 27] Create a presentation based on our approach, the findings above, our current prgress with the model, what we plan to do (port data, test cases, etc), and an outline of the paper

[by Apr 30] Should have all of our test cases done and even tested on the toy/fake data to see if we can gather valuable results from this

[by May 5] Port real data into this. Port in the vegetation and wind data and actually compute the A, B, C coeffs, and essentially have the entire coding portion completely done. Any visualization code and the actual GUI for dashboard (w/e i is) should be done by this point as well

[by May 7] Finish the final writeup and submit

####To do post presentation
Incorporate wind ✓

Incorporate safety of firefighters

Strategies for fighting fires

Figuring out optimal strategies (policy/value iteration) ✓ 
    to be done: (figure out how to represent our state space as a 2d mdp)

Bring in real data


####Paper component


## Go Time

In [2]:
import numpy as np

In [3]:
class FireModel:
    def __init__(self, fireLayerTemp, fuelSupplyMass, thermDiffus, tempRise, propCoef, scaledCoef, fuelDispearRate, ambTemp, windSpeed):
        self.fireLayerTemp = fireLayerTemp
        self.fuelSupplyMass = fuelSupplyMass
        self.thermDiffus = thermDiffus
        self.tempRise = tempRise
        self.propCoef = propCoef
        self.scaledCoef = scaledCoef
        self.fuelDispearRate = fuelDispearRate
        self.ambTemp = ambTemp
        self.windSpeed = windSpeed

In [4]:
def changeTemp():
    diffusion = np.gradient(self.thermDiffus * np.gradient(self.fireLayerTemp))
    heatAdvancedByWind = self.windSpeed * np.gradient(self.fireLayerTemp)
    rateFuelComsumedByBurning = self.fuelSupplyMass * np.exp(-self.propCoef/(self.fireLayerTemp - self.ambTemp))
    convectiveHeatLostAtmosphere = self.scaledCoef * (self.fireLayerTemp - self.ambTemp)
    return diffusion - heatAdvancedByWind + self.tempRise * (rateFuelComsumedByBurning - convectiveHeatLostAtmosphere)


In [5]:
def changeFuelSupply():
    expTemp = np.exp(-self.propCoef/(self.fireLayerTemp - self.ambTemp))
    return -1 * self.fuelDispearRate * self.fuelSupplyMass * expTemp

In [6]:
# Constants
k = 0.21360 #thermDiffus
A = 187.93 #tempRise
B = 558.49 #propCoeff
C = .000048372 #scaledCoeff
Cs = .1625 #fuelDispearRate
Tc = 1200 # Maximum Stable Combustion Temperature
Ti = 670 # Unstable Equilibrium Point

# Data-dependent variables
T = 10. #fireLayerTemp
S = 0.3 #fuelSupplyMass
Ta = 189 #ambTemp
v = 17.83 #windSpeed

## Cellular Automata

http://en.wikipedia.org/wiki/Forest-fire_model

In [105]:
import random
from utils import *
import itertools

In [120]:
## class representation for each cell
class Cell:
    def __init__(self, x, y, time, veg_inten, wind_direc, wind_inten, fire_inten, firefighter, ff_info,exting):
        self.x = x
        self.y = y
        self.time = time
        self.veg_inten = veg_inten # vegetation intensity (assuming the most intense, the more flammable; 0 means it can no longer catch on fire)
        #self.veg_vol = veg_vol #vegetation volume
        self.wind_direc = wind_direc # wind direction
        self.wind_inten = wind_inten # wind intensity
        self.fire_inten = fire_inten # if there is a fire in the cell, how intensly is it burning
        self.firefighter = firefighter
        self.ff_info = ff_info ## but will be class firefighter
        self.exting = exting

In [121]:
## code borrowed from: http://aima.cs.berkeley.edu/python/mdp.html
class MDP:
    """A Markov Decision Process, defined by an initial state, transition model,
    and reward function. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text.  Instead of T(s, a, s') being  probability number for each
    state/action/state triplet, we instead have T(s, a) return a list of (p, s')
    pairs.  We also keep track of the possible states, terminal states, and
    actions for each state. [page 615]"""

    def __init__(self, init, L, actlist, gamma=.9):
        self.L = L
        update(self, init=init, actlist=actlist,
               gamma=gamma, states=set(), reward={})

    def R(self, state):
        "Return a numeric reward for this state."
        return self.reward[state]

    def T(state, action):
        """Transition model.  From a state and an action, return a list
        of (result-state, probability) pairs."""
        abstract

    def actions(self, state):
        """Set of actions that can be performed in this state.  By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""
        ### just return all the actions for now but would have to check whether in the boundaries
        possible_actions = []
        for dx, dy in self.actlist:
            if state[0] + dx >= 0 and state[0] + dx < self.L \
               and state[1] + dy >= 0 and state[1] + dy < self.L:
                    possible_actions.append((dx,dy))
        return possible_actions
        

class GridMDP(MDP):
    """A two-dimensional grid MDP, as in [Figure 17.1].  All you have to do is
    specify the grid as a list of lists of rewards; use None for an obstacle
    (unreachable state).  Also, you should specify the terminal states.
    An action is an (x, y) unit vector; e.g. (1, 0) means move east."""
    def __init__(self, grid, actions, init=(0, 0), gamma=.9): # eliminated terminals
        grid.reverse() ## because we want row 0 on bottom, not on top
        L = len(grid)
        MDP.__init__(self, init, L, actlist=actions, gamma=gamma)
        update(self, grid=grid, rows=len(grid), cols=len(grid[0]))
        for x in range(self.cols):
            for y in range(self.rows):
                self.reward[x, y] = grid[y][x]
                if grid[y][x] is not None:
                    self.states.add((x, y))

    # think more about what our transition model here is
    def T(self, state, action):
       
        if action == None:
            return [(0.0, state)]
        else:
            # deterministic
            return [(1., (state[0]+action[0], state[1]+action[1]))]

def value_iteration(mdp, epsilon=0.001):
    "Solving an MDP by value iteration. [Fig. 17.4]"
    U1 = dict([(s, 0) for s in mdp.states])
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    while True:
        U = U1.copy()
        delta = 0
        for s in mdp.states:
            U1[s] = R(s) + gamma * max([sum([p * U[s1] for (p, s1) in T(s, a)]) for a in mdp.actions(s)])
            delta = max(delta, abs(U1[s] - U[s]))
        if delta < epsilon * (1 - gamma) / gamma:
             return U

def best_policy(mdp, U):
    """Given an MDP and a utility function U, determine the best policy,
    as a mapping from state to action. (Equation 17.4)"""
    pi = {}
    for s in mdp.states:
        pi[s] = argmax(mdp.actions(s), lambda a:expected_utility(a, s, U, mdp))
    return pi

def expected_utility(a, s, U, mdp):
    "The expected utility of doing a in state s, according to the MDP and U."
    return sum([p * U[s1] for (p, s1) in mdp.T(s, a)])


def policy_iteration(mdp):
    "Solve an MDP by policy iteration [Fig. 17.7]"
    U = dict([(s, 0) for s in mdp.states])
    pi = dict([(s, random.choice(mdp.actions(s))) for s in mdp.states])
    while True:
        U = policy_evaluation(pi, U, mdp)
        unchanged = True
        for s in mdp.states:
            a = argmax(mdp.actions(s), lambda a: expected_utility(a,s,U,mdp))
            if a != pi[s]:
                pi[s] = a
                unchanged = False
        if unchanged:
            return pi

def policy_evaluation(pi, U, mdp, k=20):
    """Return an updated utility mapping U from each state in the MDP to its
    utility, using an approximation (modified policy iteration)."""
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    for i in range(k):
        for s in mdp.states:
            U[s] = R(s) + gamma * sum([p * U[s] for (p, s1) in T(s, pi[s])])
    return U

In [122]:
## class representation for each firefighter
class FireFighter:
    def __init__(self, x, y, area, efficacy = 1):
        self.x = x # The starting cell/cell we're currently in
        self.y = y
        self.area = area # The grid where the firefighter lives
        self.path = [] # The path we've traverse so far
        self.efficacy = efficacy # Perhaps make this model vary on fighter skill level
        self.actList = [(-1,-1), (-1,0), (-1,1), (0,-1), 
                   (0, 1), (1,-1),  (1,0),  (1,1)]
                 
    def calculate_rewards(self,grid):
        # TODO: take into account safety more
        L = int(math.sqrt(len(grid)))
        reward_grid = []
        for y in range(L):
            reward_row = []
            for x in range(L):
                #if fire intensity > .8 reward for going there is 0
                if grid[(x,y)].fire_inten > .8:
                    reward_row.append(0.)
                # else if fire intensity between 0 and .8 reward for going there is fire.inten itself
                elif grid[(x,y)].fire_inten > 0 and grid[(x,y)].fire_inten <= .8:
                    reward_row.append(grid[(x,y)].fire_inten)
                # else it depends of proximity to a the most intense fire divided by how far it is
                else:
                    max_neighbor = 0
                    times = 0.
                    while max_neighbor == 0 and times < L:
                        times +=1.
                        for dx,dy in self.actList:
                            if (x+dx*times,y+dy*times) in grid:
                                max_neighbor = max(max_neighbor, grid[(x+dx*times,y+dy*times)].fire_inten)
                    reward_row.append(max_neighbor/times/2) ## need to discount somehow -- picked 1/2 arbitrarily
            reward_grid.append(reward_row)
        return reward_grid
                       
    # use value/policy interation to figure out best action   
    def bestAction2(self, newgrid):
        reward_grid = self.calculate_rewards(newgrid)
        print "reward grid", reward_grid
        my_grid = GridMDP(reward_grid, self.actList)
        values = value_iteration(my_grid)
        best_pol = best_policy(my_grid,values)
        return best_pol[(self.x,self.y)]
    
    def bestAction(self, grid):
        # Greedy
        max_inten = 0.
        max_move = None
        for dx, dy in self.actList:
            if self.x + dx >= 0 and self.x + dx < self.area.L \
               and self.y + dy >= 0 and self.y + dy < self.area.L:
                it = self.area.grid[(self.x + dx, self.y + dy)].fire_inten
                fft = grid[(self.x + dx, self.y + dy)].firefighter
                if it > max_inten and not fft:
                    max_inten = it
                    max_move = (dx, dy)
        act = max_move
        while not act:
            # Random
            act = random.choice(self.actList)
            dx, dy = act
            if self.x + dx >= 0 and self.x + dx < self.area.L \
               and self.y + dy >= 0 and self.y + dy < self.area.L \
               and not grid[(self.x + dx, self.y + dy)].firefighter:
                break
            else:
                act = None
        self.path.append(act)
        self.x += act[0]
        self.y += act[1]
        return act

In [131]:
class AreaSimulation:
    def __init__(self, L):
        self.grid = {}
        self.hood = ((-1,-1), (-1,0), (-1,1),
                (0,-1),          (0, 1),
                (1,-1),  (1,0),  (1,1))
        
        self.L = L
        self.time = 0
        self.firefighters = [[False for i in range(L)] for j in range(L)]
        self.num_fires = 0
        
        
    def fight_fire(self, ff_info):
        self.grid[(ff_info.x, ff_info.y)].firefighter = True
        self.grid[(ff_info.x, ff_info.y)].ff_info = ff_info
        self.firefighters[ff_info.x][ff_info.y] = True
        return
        
    def initialize(self):
        for x in range(self.L):
            for y in range(self.L):
                self.grid[(x,y)] = Cell(
                    x= x,
                    y= y,
                    time= 0,
                    veg_inten= random.random(),
                    #veg_vol = random.random()*10,
                    wind_direc= random.choice(["n", "nw", "w", "s", "sw", "e", "ne", "se"]),
                    wind_inten= random.random(),
                    fire_inten= 0.00,
                    firefighter = False,
                    ff_info = None,
                    exting = False
                )            
        return self.grid
    
    def wind_influence(self,x,y):
        wind_inf = 0.
        for dx,dy in self.hood:
            if (x+dx,y+dy) in self.grid:
                if ((dx == -1 and dy == -1 and self.grid[(x+dx,y+dy)].wind_direc == "se") or 
                (dx == -1 and dy == 0 and self.grid[(x+dx,y+dy)].wind_direc == "s") or 
                (dx == -1 and dy == 1 and self.grid[(x+dx,y+dy)].wind_direc == "sw") or
                (dx == 0 and dy == -1 and self.grid[(x+dx,y+dy)].wind_direc == "e") or
                (dx == 0 and dy == 1 and self.grid[(x+dx,y+dy)].wind_direc == "w") or
                (dx == 1 and dy == -1 and self.grid[(x+dx,y+dy)].wind_direc == "ne") or
                (dx == 1 and dy == 0 and self.grid[(x+dx,y+dy)].wind_direc == "n") or
                (dx == 1 and dy == 1 and self.grid[(x+dx,y+dy)].wind_direc == "nw")):
                    wind_inf += self.grid[(x+dx,y+dy)].wind_inten * self.grid[(x+dx,y+dy)].fire_inten
        return wind_inf
    
    def gnew(self):
        newgrid = {}
        new_ff_coord = []
        self.time += 1
        # iterate through all the cells
        for x in range(self.L):
            for y in range(self.L): 
                if self.firefighters[x][y]: ## there is a firefighter in that cell
                    print "firefighter in this cell", x,y
                    new_ff_coord.append((x, y))
                    self.num_fires-=1
                    
                    ## extinguish fire in this cell
                    newgrid[(x,y)] = Cell(
                        x= x,
                        y= y,
                        time = self.time if not self.grid[(x,y)].time else self.grid[(x,y)].time,
                        veg_inten = 0., # so it can't light on fire again
                        #veg_vol = max(0.,self.grid[(x,y)].veg_vol - (new_fire_inten/4.)),
                        wind_direc= self.grid[(x,y)].wind_direc,
                        wind_inten= self.grid[(x,y)].wind_inten,
                        # some previous intensity and a combination of surrounding intensities 
                        fire_inten= 0.,
                        firefighter = False,
                        ff_info = None,
                        exting = True
                   )

                elif self.grid[(x,y)].fire_inten > 0: # cell is burning
                    # if we multiply by 1 means just stays the same
                    wind_inf = 1 if self.wind_influence(x,y) == 0 else self.wind_influence(x,y)*10
                                     
                    ### rewise this fire intensity func
                    new_fire_inten = min(self.grid[(x,y)].fire_inten * (self.grid[(x,y)].veg_inten) * wind_inf * 3, 1) 
                    if new_fire_inten == 0:
                       
                        self.num_fires-=1
                    newgrid[(x,y)] = Cell(
                        x= x,
                        y= y,
                        time = self.time if not self.grid[(x,y)].time else self.grid[(x,y)].time,
                        veg_inten = max(0, self.grid[(x,y)].veg_inten - .005),
                        #veg_vol = max(0.,self.grid[(x,y)].veg_vol - (new_fire_inten/4.)),
                        wind_direc= self.grid[(x,y)].wind_direc,
                        wind_inten= self.grid[(x,y)].wind_inten,
                        # some previous intensity and a combination of surrounding intensities 
                        fire_inten= round(new_fire_inten,2),
                        firefighter = False,
                        ff_info = None,
                        exting = self.grid[(x,y)].exting
                   )
                elif self.grid[(x,y)].fire_inten == 0: # cell is not burning but can catch (assuming cells don't randomly catch fire)
                    ## need to take into account the conditions of surrounding cells
                    wind_inf = 1 if self.wind_influence(x,y) == 0 else self.wind_influence(x,y)*10
                    
                    total_inten = 0.
                    for dx,dy in self.hood:
                        if (x+dx,y+dy) in self.grid:
                            total_inten += self.grid[(x+dx,y+dy)].fire_inten # will somehow need to handle the surrounding fire  
                    new_fire_inten = (total_inten * self.grid[(x,y)].veg_inten)/8. * wind_inf
                    if new_fire_inten > 0:
                        if not self.grid[(x,y)].exting:
                            self.num_fires+=1
                            
                    #if not self.grid[(x,y)].veg_inten and new_fire_inten: 
                    newgrid[(x,y)] = Cell(
                        x= x,
                        y= y,
                        time= self.grid[(x,y)].time,
                        veg_inten = self.grid[(x,y)].veg_inten,
                        #veg_vol = max(0.,self.grid[(x,y)].veg_vol - new_fire_inten),
                        wind_direc= self.grid[(x,y)].wind_direc,
                        wind_inten= self.grid[(x,y)].wind_inten,
                        fire_inten= round(new_fire_inten,2),
                        firefighter = False,
                        ff_info = None,
                        exting = self.grid[(x,y)].exting
                   )
                    

        for x, y in new_ff_coord:
            ## put the firefighter in the next cell
            # should we pass grid or newgrid here?
            dx, dy = self.grid[(x,y)].ff_info.bestAction2(newgrid) ## somewhere here need to check if it's a legal action
            newgrid[(x+dx, y+dy)].firefighter = True
            newgrid[(x+dx, y+dy)].ff_info = self.grid[(x,y)].ff_info
            ## will need to change actual coordinates here
            newgrid[(x+dx, y+dy)].ff_info.x = x+dx
            newgrid[(x+dx, y+dy)].ff_info.y = y+dy
                
            
            ## I don't see why we need this but ok
            self.firefighters[x+dx][y+dy] = True
            self.firefighters[x][y] = False
        self.grid = newgrid
        return newgrid

    ## printing function
    def gprint(self):
        txt = '\n'.join(' * '.join(str(self.grid[(x,y)].fire_inten) for x in range(self.L))
                         for y in range(self.L))
        print(txt)
 

#### strategies. more here: http://mentalfloss.com/article/57094/10-strategies-fighting-wildfires
1. CONTROL LINE
2. BURNING OUT
3. BACKBURN
4. FLANKING
5. HOT SPOTTING
6. KNOCK DOWN
7. COLD TRAILING
8. AERIAL ATTACK
9. FIRELINE EXPLOSIVES
10. MOP-UP

In [132]:
# need to evaluate how well of a job the firefighters did
# for now: the faster that the fire gets put out
def best_ff_config(simulation,number_ffs=8,iters=50): # standard crew is 8 people
    # find all the cells where fire_inten > 0
    fires = []
    for x in range(simulation.L):
        for y in range(simulation.L):
            if simulation.grid[(x,y)].fire_inten:
                fires.append((x,y))
                
    # find all the possible places we could place the firefighters
    # (for now assuming we can't place them anywhere that's burning)
    ff_placement_poss = []
    for fire in fires:
        for dx, dy in simulation.hood:
                # if it's within the boundaries
                if fire[0] + dx >= 0 and fire[0] + dx < simulation.L \
                   and fire[1] + dy >= 0 and fire[1] + dy < simulation.L:
                        # and it's not already in our list
                        if (fire[0] + dx,fire[1] + dy) not in ff_placement_poss \
                            and (fire[0] + dx,fire[1] + dy) not in fires:
                            ff_placement_poss.append((fire[0] + dx,fire[1] + dy))
                            
    print ff_placement_poss
    best_placement = None
    best_time = float('inf')
    possible_initial_placements = list(itertools.combinations(ff_placement_poss,number_ffs))
    for placement in possible_initial_placements:
        # place all the firefighters
        for pos in placement:
            sim = copy.deepcopy(simulation)
            ff = FireFighter(pos[0],pos[1],sim)
            sim.fight_fire(ff)
        # run the simulation until all the fire is eliminated
        for i in range(iters):
            sim.gnew()
            if sim.num_fires == 0: # all have been extinguished
                if i < best_time: # log the best time
                    best_time = i
                    best_placement = placement
    return best_placement
   

    
sim = AreaSimulation(4)
sim.initialize()

## start a fire
sim.grid[(3,2)].fire_inten = .5
sim.grid[(2,1)].fire_inten = .6
sim.num_fires = 2

print best_ff_config(sim,3)# place 3 firefighters in the simulation

[(1, 0), (1, 1), (1, 2), (2, 0), (2, 2), (3, 0), (3, 1), (2, 3), (3, 3)]
firefighter in this cell 1 2
reward grid [[0.025, 0.05, 0.5, 0.07], [0.025, 0.01, 0.0, 0.11], [0.005, 0.5, 0.03, 0.0], [0.25, 0.015, 0.5, 0.01]]
firefighter in this cell 1 1
reward grid [[0.05, 0.1, 0.01, 0.18], [0.01, 0.5, 0.0, 0.5], [0.005, 0.5, 0.02, 0.0], [0.25, 0.01, 0.01, 0.5]]
firefighter in this cell 1 2
reward grid [[0.01, 0.21, 0.5, 0.47], [0.02, 0.5, 0.0, 0.5], [0.01, 0.5, 0.01, 0.0], [0.25, 0.005, 0.5, 0.02]]
firefighter in this cell 1 1
reward grid [[0.03, 0.43, 0.02, 0.0], [0.215, 0.5, 0.0, 0.5], [0.0075, 0.5, 0.01, 0.0], [0.25, 0.005, 0.01, 0.01]]
firefighter in this cell 1 2
reward grid [[0.2, 0.0, 0.01, 0.0], [0.01, 0.5, 0.0, 0.5], [0.005, 0.5, 0.01, 0.0], [0.25, 0.005, 0.5, 0.5]]
firefighter in this cell 1 1
reward grid [[0.0, 0.0, 0.5, 0.0], [0.03, 0.5, 0.0, 0.5], [0.015, 0.5, 0.01, 0.0], [0.25, 0.005, 0.01, 0.02]]
firefighter in this cell 1 2
reward grid [[0.0, 0.0, 0.03, 0.0], [0.4, 0.5, 0.0, 

In [128]:
sim = AreaSimulation(4)
sim.initialize()

## start a fire
sim.grid[(3,2)].fire_inten = .5
sim.grid[(2,1)].fire_inten = .6
sim.num_fires = 2
sim.gprint()
iters = 5
for i in range(iters):
    print "iteration",i
    if i == 0:
        # introduce some firefighter
        ff1 = FireFighter(3,2,sim)
        sim.fight_fire(ff1)
    sim.gprint()
    sim.gnew()
    print "num fires",sim.num_fires

0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.6 * 0.0
0.0 * 0.0 * 0.0 * 0.5
0.0 * 0.0 * 0.0 * 0.0
iteration 0
0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.6 * 0.0
0.0 * 0.0 * 0.0 * 0.5
0.0 * 0.0 * 0.0 * 0.0
cell just caught on fire! 1 0
cell just caught on fire! 1 1
cell just caught on fire! 1 2
cell just caught on fire! 2 0
cell just caught on fire! 2 2
cell just caught on fire! 2 3
cell just caught on fire! 3 0
cell just caught on fire! 3 1
firefighter in this cell 3 2
cell extinguished 3 2
cell just caught on fire! 3 3
reward grid [[0.035, 0.03, 0.04, 0.04], [0.145, 0.07, 0.46, 0.14], [0.145, 0.29, 0.1, 0.23], [0.145, 0.145, 0.04, 0.06]]
num fires 10
iteration 1
0.0 * 0.03 * 0.04 * 0.04
0.0 * 0.07 * 0.46 * 0.14
0.0 * 0.29 * 0.1 * 0.0
0.0 * 0.0 * 0.04 * 0.06
cell just caught on fire! 0 0
cell just caught on fire! 0 1
cell just caught on fire! 0 2
cell just caught on fire! 0 3
cell just caught on fire! 1 3
firefighter in this cell 2 2
cell extinguished 2 2
reward grid [[0.095, 0.04, 0.06, 0.07], [0.01, 


#### TO-DO:

simulation.num_fires not counted correctly

improve calculate rewards function (very naive)
